<a href="https://colab.research.google.com/github/HaithemH/Coronavirus-death-rate-prediction/blob/master/Background_video_substitution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [35]:
!git clone https://github.com/lev1khachatryan/Video_Background_Editor

Cloning into 'Video_Background_Editor'...
remote: Enumerating objects: 198, done.
remote: Counting objects: 100% (145/145), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 198 (delta 60), reused 108 (delta 24), pack-reused 53
Receiving objects: 100% (198/198), 53.86 MiB | 31.23 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [36]:
!rm -r /content/Video_Background_Editor


In [1]:
!pip install -r /content/Video_Background_Editor/requirements.txt


In [15]:
!python3 /content/Video_Background_Editor/python/mo_subtractor.py

[ERROR:0] global /io/opencv/modules/videoio/src/cap.cpp (392) open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.1.2) /io/opencv/modules/videoio/src/cap_images.cpp:253: error: (-5:Bad argument) CAP_IMAGES: can't find starting number (in the name of file): /content/Video_Background_Editor/_output/output.avi in function 'icvExtractPattern'




In [101]:
#background extraction
from __future__ import print_function
import numpy as np
import cv2
import configparser
import json

def extract_background(background_video):
	
  #print(background_video)
  cap = cv2.VideoCapture(background_video)
  
  if not cap.isOpened:
	    print('Unable to open: ' + BACKGROUND_PATH + BG_VIDEO_NAME)
	    exit(0)

	# Randomly select 25 frames
  frameIds = cap.get(cv2.CAP_PROP_FRAME_COUNT) * np.random.uniform(size=80)
   
	# Store selected frames in an array
  frames = []
  for fid in frameIds:
    cap.set(cv2.CAP_PROP_POS_FRAMES, fid)
    ret, frame = cap.read()
    frames.append(frame)
   
	# Calculate the median along the time axis
  medianFrame = np.median(frames, axis=0).astype(dtype=np.uint8)
  


	#Display median frame
  #cv2.imshow('frame', medianFrame)
	#cv2.waitKey(0)

	# Save median frame to output folder
  cv2.imwrite(OUTPUT_PATH + 'background.jpg', medianFrame)

	
	# When everything done, release the capture
  cap.release()
  cv2.destroyAllWindows()
  
  return medianFrame
  
if __name__ == '__main__':


  # Read json config file
  with open('/content/Video_Background_Editor/python/config.json', 'r') as config_file:
    config = json.load(config_file)
	
	# Get necessary information from config file
  INPUT_PATH = config['paths']['INPUT_PATH']
  OUTPUT_PATH = config['paths']['OUTPUT_PATH']
  BACKGROUND_PATH = config['paths']['BACKGROUND_PATH']
  BG_VIDEO_NAME = config['background']['backg_video_name']

  background_path=BACKGROUND_PATH + BG_VIDEO_NAME
  extract_background(background_path)

In [105]:
#moving objects extraction
from __future__ import print_function
import numpy as np
import cv2
import configparser
from os import walk
import os
# Find OpenCV version
(major_ver, minor_ver, subminor_ver) = (cv2.__version__).split('.')
def mo_sub(video,output):

#Function to substract moving objects from the backgroud

	# Open video and check if it successfully opened
  #print(video)
  cap = cv2.VideoCapture(video)
  if not cap.isOpened:
	  print('Unable to open: ' + video)
	  exit(0)

		# Get fps and shape of the video
  if int(major_ver)  < 3:
	  fps = cap.get(cv2.cv.CV_CAP_PROP_FPS)
  else:
	  fps = cap.get(cv2.CAP_PROP_FPS)
  size = (int(cap.get(3)),int(cap.get(4))) 

		# Create video writer
  fourcc = cv2.VideoWriter_fourcc(*'XVID')
  out = cv2.VideoWriter(output, fourcc, fps, size)
  

	# Create background subtractor
  backSub = cv2.createBackgroundSubtractorMOG2()
	# backSub = cv2.createBackgroundSubtractorKNN()
  
  while True:
    
    _, frame = cap.read()
    #print(frame)
    if frame is None:
      
      break
    
    fgMask = backSub.apply(frame)

    fgMask = cv2.cvtColor(fgMask,cv2.COLOR_GRAY2RGB)
	    	# fgMask = np.stack([fgMask]*3, axis = -1)
	    	# out.write(fgMask * frame)
    out.write(fgMask)

	# When everything done, release the capture
  cap.release()
  cv2.destroyAllWindows()

if __name__ == '__main__':
    # Read config file
  with open('/content/Video_Background_Editor/python/config.json', 'r') as config_file:
    config = json.load(config_file)
	
	# Get necessary information from config file
  INPUT_PATH = config['paths']['INPUT_PATH']
  OUTPUT_PATH = config['paths']['OUTPUT_PATH']
  #BG_VIDEO_NAME = config['background']['backg_video_name']
  filenames = next(walk(INPUT_PATH), (None, None, []))[2]
  #print(filenames)
  for file in filenames:
		
    f=os.path.splitext(file)[0]
    video=INPUT_PATH + file
    
    output=OUTPUT_PATH+f+'_mo.avi'
    mo_sub(video,output)
    



In [116]:
#combiner
from __future__ import print_function
import numpy as np
import cv2
import configparser
import os

# Find OpenCV version
(MAJOR_VER, MINOR_VER, SUBMINOR_VER) = (cv2.__version__).split('.')

def Combiner(path, bg_frame, save_dir):
  '''
	Change background in videos using GrabCut algorithm
	------------------------
	Parameters
		path (string): path of the video to change the background
		bg_frame (array like): background image (or nd.array)
		save_dir (string): path where to save obtained video
	Returns
		None
	------------------------
'''

  # Get video name from full path
  file_name = path.split('/')[-1].split('.')[0]
	# Open the video and check if it successfully done
  cap = cv2.VideoCapture(path)
  if not cap.isOpened:
    print('Unable to open: ' + path)
    exit(0)

	# Get fps and shape of the video
  if int(MAJOR_VER) < 3:
    fps = cap.get(cv2.cv.CV_CAP_PROP_FPS)
  else:
    fps = cap.get(cv2.CAP_PROP_FPS)
  print(fps)
  size = (int(cap.get(3)),int(cap.get(4)))

	# Create video writer
  fourcc = cv2.VideoWriter_fourcc(*'XVID')
  out = cv2.VideoWriter(save_dir + file_name + '_new_BG.avi', fourcc, fps, size)

	# Resize background frame (image) to video shape
  background = cv2.resize(bg_frame, size, interpolation=cv2.INTER_AREA)
	
	# Loop over the frames of the video
  while True:
		# Grab the current frame
	  _, frame = cap.read()
	    # If the frame could not be grabbed, then we have reached the end of the video
	  if frame is None:
	   	break

	    # Convert it to grayscale
	  frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
	    # Initial mask
	  mask = np.zeros(frame.shape[:2], np.uint8)
	    # These are arrays used by the algorithm internally.
	  bgdModel = np.zeros((1, 65), np.float64)
	  fgdModel = np.zeros((1, 65), np.float64)
	    # Specify a region of interest (RoI) and apply grabCut algorithm
	  rect = (200, 50, 300, 400)
	    # Number of iterations the algorithm should run is 1
	    # which is fast but not good for correct segmentation
	  cv2.grabCut(frame, mask, rect, bgdModel, fgdModel, 1, cv2.GC_INIT_WITH_RECT)
	    # New mask for moving object
	  mask2 = np.where((mask == 2) | (mask == 0), (0,), (1,)).astype('uint8')
	  frame = frame * mask2[:, :, np.newaxis]
	  mask_1 = frame > 0
	  mask_2 = frame <= 0
	    # Linear combination of bgd and fgd frames with mask_1 and mask_2 "scaliars"
	  combination = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) * mask_1 + background * mask_2
	  combination = combination.astype(dtype=np.uint8)
	    # Write combined frame
	  out.write(combination)

	# When everything done, cleanup the camera and close any open windows
  print('Background substitution is successfully finished')
  cap.release()
  cv2.destroyAllWindows()


def main():

	# Read config file
	  # Read config file
  with open('/content/Video_Background_Editor/python/config.json', 'r') as config_file:
    config = json.load(config_file)
	
	# Get necessary information from config file
  INPUT_PATH = config['paths']['INPUT_PATH']
  OUTPUT_PATH = config['paths']['OUTPUT_PATH']
  BG_VIDEO_NAME = config['background']['backg_video_name']
  BACKGROUND_PATH = config['paths']['BACKGROUND_PATH']
  background_path=BACKGROUND_PATH + BG_VIDEO_NAME
  #extract the background from the given video
  background = extract_background(background_path)
  #print(background)

  for file in os.listdir(INPUT_PATH):
    if file.endswith('.avi'):
      path=os.path.join(INPUT_PATH, file)
      print("Precessing.................:",path)
      Combiner(path, background, OUTPUT_PATH)

if __name__ == '__main__':
    main()

Precessing.................: /content/Video_Background_Editor/_input/Haithem_moving.avi
30.0
Background substitution is successfully finished


In [100]:
from IPython.display import HTML
from base64 import b64encode

video_path = "/content/Video_Background_Editor/_output/example_01_new_BG.avi"

mp4 = open(video_path, "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=400 controls>
      <source src="{data_url}" type="video/mp4">
</video>
""")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!python3 /content/Video_Background_Editor/python/mo_subtractor.py

# Load videos from config file

# Background Extraction